<a href="https://colab.research.google.com/github/wolfram-laube/mlpc-project_team-park/blob/wl/pre-trained/fastlane.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## All-in-one

In [ ]:
data_dir = '../dataset'

### audio_filename_utils.py

In [ ]:
# audio_parsing_utils.py

import re
import unicodedata
import logging
from collections import defaultdict

# Setup logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Define the pattern to parse clipped command filenames
clipped_command_pattern = re.compile(
    r'(\d+)_speech_(true|false)_((?:[a-zA-ZäöüÄÖÜß]+_(?:an|aus)_?)+)_(\d+\.\d+)_(\d+\.\d+)\.wav', re.UNICODE
)

# Define the pattern to parse full scene filenames
full_scene_pattern = re.compile(
    r'(\d+)_speech_(true|false)_((?:[a-zA-ZäöüÄÖÜß]+_(?:an|aus)_?)+)\.wav', re.UNICODE
)

def normalize_unicode(text):
    return unicodedata.normalize('NFC', text)

# Function to parse clipped command filenames to extract commands, start time, and end time
def parse_clipped_command_filename(filename):
    logger.debug(f"Attempting to parse filename: {filename}")
    filename = normalize_unicode(filename)
    match = clipped_command_pattern.match(filename)
    if not match:
        logger.error(f"Filename {filename} does not match the expected pattern.")
        raise ValueError(f"Filename {filename} does not match the expected pattern.")

    # Extract command string and timestamps
    commands_str = match.group(3)
    start_time = float(match.group(4))
    end_time = float(match.group(5))

    # Split and format commands
    commands = commands_str.split('_')
    command_list = []
    for i in range(0, len(commands), 2):
        command_list.append(f"{commands[i]} {commands[i+1]}")

    logger.debug(f"Parsed filename {filename}: file_id={match.group(1)}, speech_flag={match.group(2)}, command_list={command_list}, start_time={start_time}, end_time={end_time}")
    return match.group(1), match.group(2), command_list, start_time, end_time

# Function to parse full scene filenames to extract commands
def parse_full_scene_filename(filename):
    logger.debug(f"Attempting to parse filename: {filename}")
    filename = normalize_unicode(filename)
    match = full_scene_pattern.match(filename)
    if not match:
        logger.error(f"Filename {filename} does not match the expected pattern.")
        raise ValueError(f"Filename {filename} does not match the expected pattern.")

    # Extract command string
    file_id = match.group(1)
    speech_flag = match.group(2)
    commands_str = match.group(3)

    # Split and format commands
    commands = commands_str.split('_')
    command_list = []
    for i in range(0, len(commands), 2):
        command_list.append(f"{commands[i]} {commands[i+1]}")

    logger.debug(f"Parsed filename {filename}: file_id={file_id}, speech_flag={speech_flag}, command_list={command_list}")
    return file_id, speech_flag, command_list


### audio_loading_utils.py

In [ ]:
# audio_loading_utils.py

import os
import librosa
import numpy as np
"""
from audio_parsing_utils import (
    parse_clipped_command_filename,
    parse_full_scene_filename
)
"""

# Function to load audio files from the scenes directory
def load_scene_files(directory):
    audio_files = []
    for filename in os.listdir(directory):
        if filename.endswith(".wav"):
            filepath = os.path.join(directory, filename)
            y, sr = librosa.load(filepath, sr=16000)
            y = y.astype(np.float32)  # Ensure all audio data is of type float32
            file_id, _, commands = parse_full_scene_filename(filename)
            audio_files.append({"path": filepath, "audio": y, "sr": sr, "text": " ".join(commands)})
    return audio_files

# Function to load audio files from the words directory
def load_word_files(directory):
    audio_files = []
    for root, _, filenames in os.walk(directory):
        for filename in filenames:
            if filename.endswith(".wav"):
                filepath = os.path.join(root, filename)
                y, sr = librosa.load(filepath, sr=16000)
                y = y.astype(np.float32)  # Ensure all audio data is of type float32
                text = os.path.basename(root)  # Extract text from folder name
                audio_files.append({"path": filepath, "audio": y, "sr": sr, "text": text})
    return audio_files

# Function to load audio files from the clipped commands directory
def load_clipped_command_files(directory):
    audio_files = []
    for filename in os.listdir(directory):
        if filename.endswith(".wav"):
            filepath = os.path.join(directory, filename)
            y, sr = librosa.load(filepath, sr=16000)
            y = y.astype(np.float32)  # Ensure all audio data is of type float32
            _, _, command_list, start_time, end_time = parse_clipped_command_filename(filename)
            command = " ".join(command_list)
            audio_files.append({"path": filepath, "audio": y, "sr": sr, "text": command, "start_time": start_time, "end_time": end_time})
    return audio_files


### data_collator.py

In [ ]:
# data_collator.py

import torch
from transformers import Wav2Vec2Processor

class DataCollatorCTCWithPadding:
    def __init__(self, processor: Wav2Vec2Processor, padding=True):
        self.processor = processor
        self.padding = padding

    def __call__(self, features):
        input_values = [feature["input_values"] for feature in features]
        labels = [feature["labels"] for feature in features]

        batch = self.processor.pad(
            {"input_values": input_values},
            padding=self.padding,
            return_tensors="pt"
        )

        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                {"input_ids": labels},
                padding=self.padding,
                return_tensors="pt"
            )

        # Replace padding with -100 to ignore them during loss computation
        labels_batch["input_ids"][labels_batch["input_ids"] == self.processor.tokenizer.pad_token_id] = -100

        batch["labels"] = labels_batch["input_ids"]

        return batch


In [ ]:
# your_script.py

# Install necessary libraries if not already installed
!pip install transformers librosa torch datasets noisereduce evaluate jiwer pandas

import os
import torch
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC, Trainer, TrainingArguments
from datasets import Dataset, DatasetDict, load_metric
"""from audio_loading_utils import (
    load_scene_files,
    load_word_files,
    load_clipped_command_files
)
from data_collator import DataCollatorCTCWithPadding"""

# Set environment variables for MPS fallback and high watermark ratio
os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"
os.environ["PYTORCH_MPS_HIGH_WATERMARK_RATIO"] = "0.0"

# Set the default tensor type to float32
torch.set_default_dtype(torch.float32)

# Define directories
data_dir = '../dataset'
scenes_dir = f'{data_dir}/scenes/wav'
words_dir = f'{data_dir}/words'
clipped_commands_dir = f'{data_dir}/clipped_commands'

# Load datasets
scenes_data = load_scene_files(scenes_dir)
words_data = load_word_files(words_dir)
clipped_commands_data = load_clipped_command_files(clipped_commands_dir)

# Function to create a dataset from the audio files
def create_dataset(audio_files):
    data = {"path": [], "audio": [], "text": []}
    for item in audio_files:
        data["path"].append(item["path"])
        data["audio"].append(item["audio"].tolist())  # Convert numpy array to list
        data["text"].append(item["text"])
    return Dataset.from_dict(data)

# Create datasets
scenes_dataset = create_dataset(scenes_data)
words_dataset = create_dataset(words_data)
clipped_commands_dataset = create_dataset(clipped_commands_data)

# Combine datasets into a DatasetDict
dataset = DatasetDict({"train": scenes_dataset, "test": words_dataset})

# Load the pre-trained model and processor
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base")

# Move model to CPU
device = torch.device("cpu")
model.to(device)
print(f"Using device: {device}")

# Preprocess function for dataset
def preprocess(batch):
    audio = batch["audio"]
    batch["input_values"] = processor(audio, sampling_rate=16000).input_values[0]
    batch["labels"] = processor.tokenizer(batch["text"], padding="max_length", max_length=128, truncation=True).input_ids
    return batch

# Apply preprocessing
dataset = dataset.map(preprocess, remove_columns=["path", "audio", "text"])

# Define data collator
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

# Define metric
wer_metric = load_metric("wer")

def compute_metrics(pred):
    pred_ids = pred.predictions.argmax(-1)
    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id
    pred_str = processor.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = processor.batch_decode(pred.label_ids, skip_special_tokens=True)
    wer = wer_metric.compute(predictions=pred_str, references=label_str)
    return {"wer": wer}

# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,  # Reduce batch size
    per_device_eval_batch_size=4,  # Reduce batch size
    num_train_epochs=3,
    weight_decay=0.01,
    save_steps=10,
    save_total_limit=2,
    logging_dir="./logs",
    logging_steps=10,
    report_to="none",  # to disable wandb and other integrations
    gradient_accumulation_steps=2,  # Accumulate gradients
    fp16=False,  # Ensure mixed precision training is disabled
    bf16=False,  # Ensure BF16 precision is disabled
    no_cuda=True  # Explicitly disable CUDA
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    tokenizer=processor.feature_extractor,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

# Train the model
trainer.train()

# Evaluate the model
eval_results = trainer.evaluate()
print(f"Evaluation results: {eval_results}")


/usr/local/lib/python3.11/site-packages/transformers/configuration_utils.py:364: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base and are newly initialized: ['lm_head.bias', 'lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Using device: cpu


Map:   0%|          | 0/814 [00:00<?, ? examples/s]

Map:   0%|          | 0/45296 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/site-packages/datasets/load.py:759: FutureWarning: The repository for wer contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.2/metrics/wer/wer.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
/usr/local/lib/python3.11/site-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.11/site-packages/transformers/training_args.py:1489: FutureWarning: using `no_cuda` is deprecated and will be removed in version 5.0 of 🤗 Transformers. Use `use_cpu` instead
  warnings.warn(
/usr/local/lib/python3.11/site-packages/tran

Epoch,Training Loss,Validation Loss


/usr/local/lib/python3.11/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:156: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.11/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:156: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(


In [ ]:
model_dir = f"{data_dir}/meta"
processor_dir = f"{data_dir}/meta"

# Check if existing model and processor exist
if os.path.exists(model_dir) and os.path.exists(processor_dir):
    # Load the existing model and processor
    existing_model = Wav2Vec2ForCTC.from_pretrained(model_dir)
    existing_processor = Wav2Vec2Processor.from_pretrained(processor_dir)
    existing_model.to(device)

    # Initialize a new Trainer for the existing model
    existing_trainer = Trainer(
        model=existing_model,
        args=training_args,
        train_dataset=dataset["train"],
        eval_dataset=dataset["test"],
        tokenizer=existing_processor.feature_extractor,
        data_collator=data_collator,
        compute_metrics=compute_metrics
    )

    # Evaluate the existing model
    existing_eval_results = existing_trainer.evaluate()
    print(f"Existing model evaluation results: {existing_eval_results}")

    # Compare WER (lower is better)
    if new_eval_results["eval_wer"] < existing_eval_results["eval_wer"]:
        print("New model performs better. Saving new model and processor.")
        model.save_pretrained(model_dir)
        processor.save_pretrained(processor_dir)
    else:
        print("Existing model performs better. Keeping existing model and processor.")
else:
    print("No existing model found. Saving new model and processor.")
    model.save_pretrained(model_dir)
    processor.save_pretrained(processor_dir)

In [ ]:
# inference_script.py

import os
import torch
import librosa
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor

# Define the directory where the model and processor are saved
model_dir = "path/to/save/model"
processor_dir = "path/to/save/processor"

# Load the saved model and processor
model = Wav2Vec2ForCTC.from_pretrained(model_dir)
processor = Wav2Vec2Processor.from_pretrained(processor_dir)

# Move model to CPU
device = torch.device("cpu")
model.to(device)

# Function to transcribe an audio file with timestamps
def transcribe(audio_path):
    # Load audio file
    audio_input, sr = librosa.load(audio_path, sr=16000)

    # Preprocess the audio input
    input_values = processor(audio_input, return_tensors="pt", padding="longest").input_values.to(device)

    # Perform inference
    with torch.no_grad():
        logits = model(input_values).logits

    # Decode the logits to text
    predicted_ids = torch.argmax(logits, dim=-1)
    transcription = processor.batch_decode(predicted_ids)[0]

    # Get the timestamps
    timestamps = []
    input_lengths = input_values.shape[1]
    time_per_input = len(audio_input) / sr / input_lengths
    last_token = None

    for i, token in enumerate(predicted_ids[0]):
        if token != processor.tokenizer.pad_token_id and token != last_token:
            word = processor.tokenizer.decode([token])
            start_time = i * time_per_input
            end_time = (i + 1) * time_per_input
            timestamps.append((word, start_time, end_time))
            last_token = token

    return transcription, timestamps

# Example usage
audio_path = "path/to/audio/file.wav"
transcription, timestamps = transcribe(audio_path)
print(f"Transcription: {transcription}")
for word, start, end in timestamps:
    print(f"Word: {word}, Start time: {start:.2f}, End time: {end:.2f}")
